In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
test_df = pd.read_csv('test.csv')

In [3]:
# Remove rows with missing values and duplicates in testing data
test_df.dropna(inplace=True)
test_df.drop_duplicates(inplace=True)

In [5]:
# Ensure only relevant columns are kept
test_df = test_df[['label', 'comment']]

In [11]:
def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    
    return text

test_df['cleaned_comment'] = test_df['comment'].apply(clean_text)

In [13]:
# Balancing the testing dataset
count_1s = test_df['label'].value_counts()[1]
count_0s = test_df['label'].value_counts()[0]
diff = abs(count_1s - count_0s)

In [21]:
print(count_1s)

8379


In [23]:
print(count_0s)

11299


In [14]:

if count_1s > count_0s:
    # Remove extra 1s
    data_1s = test_df[test_df['label'] == 1]
    data_0s = test_df[test_df['label'] == 0]
    data_1s_balanced = data_1s.sample(n=count_0s)
    balanced_test_df = pd.concat([data_1s_balanced, data_0s])
else:
    # Remove extra 0s
    data_1s = test_df[test_df['label'] == 1]
    data_0s = test_df[test_df['label'] == 0]
    data_0s_balanced = data_0s.sample(n=count_1s)
    balanced_test_df = pd.concat([data_1s, data_0s_balanced])

In [15]:
# Save the cleaned and balanced datasets
balanced_test_df.to_csv('cleaned_balanced_test_data.csv', index=False)

In [19]:
# Display the first few rows of the cleaned and balanced testing data
print("Cleaned and Balanced Testing Data:")
print(balanced_test_df.head())

Cleaned and Balanced Testing Data:
     label                                            comment  \
58     1.0                                Fuck anyone from NJ   
87     1.0                 But Snowbowl is opening in 9 days!   
98     1.0                It's a metaphorical *physical* wall   
109    1.0  Pretty sure Malcolm already gave you the solut...   
132    1.0                           Who is Mike Huckleberry?   

                                       cleaned_comment  
58                                      fuck anyone nj  
87                             snowbowl opening 9 days  
98                          metaphorical physical wall  
109  pretty sure malcolm already gave solution move...  
132                                   mike huckleberry  


In [22]:
# Display the balanced class distribution in testing data
print("\nBalanced Dataset:")
print(balanced_test_df['label'].value_counts())


Balanced Dataset:
label
1.0    8379
0.0    8379
Name: count, dtype: int64
